In [1]:
from amdex import AMdEX
import hashlib

BASE_URL = "https://develop.amdex.dev"

name = "Predictive maintenance server"
password = "yes"
password_hash = hashlib.sha256(password.encode()).hexdigest().upper()

# Create an instance of AMdEX class
amdex = AMdEX()
    

In [2]:
# Authenticate
amdex.authenticate(name, password_hash)
# amdex.authenticate2(name, password_hash)

got base url https://develop.amdex.dev


In [3]:
# Get account info
account_info = amdex.getAccountInfo()
print("Account Info:", account_info)

Account Info: {'name': 'Predictive maintenance server', 'public_key': 'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA2WgGfdnqUL0qwQ0sG3LVuAVX9JST4/W5yvAjnJlPEC33YVagYghQrMqhg/QCku2wSXCsztuwAaf9ugqZemheP8RfYeQkbm0d6wzXpOj6R7wBBV7j+aFAkbBlAKNKWyDAvn2HcS1jHNtcNfJQsaez6P5B1pC7OJdi+w1cHdgwLxIz0Kdm2nSYhUrnOZtDyl4hV4kwSFHiaFpIpihyTcbQhlnyNjE75auu6cL8G/wFBo/LPVNZmirWWa1Dvwfle20hc5AOsIFbMeHMnoVbsGFHBRO3eBMVdjGFrxv6ITW0juqpoHypjenVR4G3soklInr2Z4koHRBhM1yPoG3cpkuzMQIDAQAB', 'auditlog_write_access': True, 'auditlog_read_access_for_entries_by_account_uuids': ['6ae1ae4f-3c5c-4d55-8785-86076c6586f9', '886fc572-2522-4bf1-9399-3fca39fa946c', '35e825ef-739f-4c02-8e75-c8697021fc2e', 'cc633679-e4a6-4033-8b62-26ecc11ddb8e'], 'account_uuid': 'cc633679-e4a6-4033-8b62-26ecc11ddb8e', 'bearer_token': 'cc633679-e4a6-4033-8b62-26ecc11ddb8e'}


Get Version

In [4]:
# Get version
version_info = amdex.version.get()
print("Version:", version_info)

In [5]:
# Get dataviews
dataviews = amdex.dataviews.get()
print("Dataviews:", dataviews)

TypeError: get expected at least 1 arguments, got 0

In [4]:
# Create a dataview
dataview_body = {
    "name": "Sample Dataview",
    "query": "SELECT * FROM table",
    # ... other properties
}
new_dataview = amdex.dataviews.post(dataview_body)
print("New Dataview:", new_dataview)

In [4]:
# Delete a dataview
dataview_uuid = "abc123"
result = amdex.dataviews.delete(dataview_uuid)
print("Delete Result:", result)

In [4]:
# Get jobs
jobs = amdex.jobs.get()
print("Jobs:", jobs)

In [4]:
# Create a job
job_body = {
    "name": "Sample Job",
    "query": "SELECT * FROM table",
    # ... other properties
}
new_job = amdex.jobs.post(job_body)
print("New Job:", new_job)

In [4]:
# Delete jobs
delete_result = amdex.jobs.delete()
print("Delete Result:", delete_result)

In [4]:
# Poll job status
job_uuid = "xyz789"
status = amdex.jobs.pollStatus(job_uuid)
print("Job Status:", status)

In [4]:
# Get audit logs
job_uuid = "xyz789"
audit_logs = amdex.auditlogs.get(job_uuid)
print("Audit Logs:", audit_logs)

In [4]:
# Create audit log
job_uuid = "xyz789"
audit_log_body = {
    "message": "Sample Audit Log",
    # ... other properties
}

In [4]:
new_audit_log = amdex.auditlogs.post(job_uuid, audit_log_body)
print("New Audit Log:", new_audit_log)

TypeError: get expected at least 1 arguments, got 0